# Option Pricing with Monte Carlo and Quasi-Monte Carlo

This notebook demonstrates option pricing using the `qmc_options` package.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from qmc_options import generators, pricing, analytical, utils

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

## 1. European Call Option

Let's start with a simple European call option and compare MC, QMC, and the analytical Black-Scholes formula.

In [ ]:
# Option parameters
params = {
    'S0': 100.0,      # Initial stock price
    'K': 100.0,       # Strike price (ATM)
    'r': 0.05,        # Risk-free rate (5%)
    'delta': 0.02,    # Dividend yield (2%)
    'sigma': 0.25,    # Volatility (25%)
    'T': 1.0          # Time to maturity (1 year)
}

# Black-Scholes analytical price
bs_price = analytical.black_scholes_call(**params)
print(f"Black-Scholes Price: ${bs_price:.4f}")

In [ ]:
# Compare MC vs QMC convergence
fib_indices = range(6, 16)
sample_sizes = []
mc_prices = []
qmc_prices = []

np.random.seed(42)

for m in fib_indices:
    # Get sample size from Fibonacci
    fib_seq = utils.fibonacci_sequence(m)
    N = fib_seq[-1]
    sample_sizes.append(N)
    
    # MC with random points
    mc_points = np.random.rand(N)
    mc_price = pricing.european_call_mc(points=mc_points, **params)
    mc_prices.append(mc_price)
    
    # QMC with Halton sequence
    qmc_points = generators.halton([2], N)[:, 0]
    qmc_price = pricing.european_call_mc(points=qmc_points, **params)
    qmc_prices.append(qmc_price)

# Create results DataFrame
results = pd.DataFrame({
    'N': sample_sizes,
    'MC Price': mc_prices,
    'QMC Price': qmc_prices,
    'MC Error': np.abs(np.array(mc_prices) - bs_price),
    'QMC Error': np.abs(np.array(qmc_prices) - bs_price)
})

print("\nConvergence Results:")
print(results.to_string(index=False))

In [ ]:
# Plot convergence
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(sample_sizes, mc_prices, 'o-', label='MC', linewidth=2)
plt.plot(sample_sizes, qmc_prices, 's-', label='QMC', linewidth=2)
plt.axhline(bs_price, color='red', linestyle='--', label='Black-Scholes', linewidth=2)
plt.xlabel('Sample Size (N)')
plt.ylabel('Option Price ($)')
plt.title('European Call Option Price Convergence')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.loglog(sample_sizes, results['MC Error'], 'o-', label='MC Error', linewidth=2)
plt.loglog(sample_sizes, results['QMC Error'], 's-', label='QMC Error', linewidth=2)
plt.xlabel('Sample Size (N)')
plt.ylabel('Absolute Error')
plt.title('Error Convergence (Log-Log)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Spread Options

Spread options have payoff: $\max(w_1 S_1 - w_2 S_2 - K, 0)$

For $K=0$, we can validate against the Margrabe formula.

In [ ]:
# Spread option parameters
spread_params = {
    'w1': 1.0,
    'w2': 1.0,
    'r': 0.05,
    'K': 0.0,  # Use K=0 to compare with Margrabe
    'S10': 100.0,
    'S20': 105.0,
    'delta1': 0.03,
    'delta2': 0.04,
    'sigma1': 0.25,
    'sigma2': 0.20,
    'rho12': 0.7,
    'T': 1.0
}

# Margrabe formula (exact for K=0)
margrabe_price = analytical.margrabe_formula(
    S10=spread_params['S10'],
    S20=spread_params['S20'],
    delta1=spread_params['delta1'],
    delta2=spread_params['delta2'],
    sigma1=spread_params['sigma1'],
    sigma2=spread_params['sigma2'],
    rho12=spread_params['rho12'],
    T=spread_params['T']
)

print(f"Margrabe Formula Price: ${margrabe_price:.4f}")

In [ ]:
# Price with QMC using Good Lattice Points
spread_errors = []
spread_sizes = []

for m in range(8, 16):
    glp = generators.good_lattice_points(m)
    N = len(glp)
    spread_sizes.append(N)
    
    qmc_price = pricing.spread_option(points=glp, **spread_params)
    error = abs(qmc_price - margrabe_price)
    spread_errors.append(error)
    
    print(f"N={N:4d}: Price=${qmc_price:.4f}, Error=${error:.6f}")

In [ ]:
# Plot spread option convergence
plt.figure(figsize=(10, 6))
plt.loglog(spread_sizes, spread_errors, 'o-', linewidth=2, markersize=8)
plt.xlabel('Sample Size (N)')
plt.ylabel('Absolute Error')
plt.title('Spread Option Pricing - QMC Convergence')
plt.grid(True, alpha=0.3)
plt.show()

## 3. Asian Options

Asian options depend on the average price of the underlying over the option's life.

In [ ]:
# Asian option parameters
asian_params = {
    'S0': 100.0,
    'K': 100.0,
    'r': 0.05,
    'delta': 0.02,
    'sigma': 0.25,
    'T': 1.0,
    'm': 12  # Monthly averaging
}

# Generate 12-dimensional Halton sequence
primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37]
N = 5000
halton_12d = generators.halton(primes, N)

asian_price = pricing.asian_call(points=halton_12d, **asian_params)
european_price = analytical.black_scholes_call(
    S0=asian_params['S0'],
    K=asian_params['K'],
    r=asian_params['r'],
    delta=asian_params['delta'],
    sigma=asian_params['sigma'],
    T=asian_params['T']
)

print(f"Asian Call Price: ${asian_price:.4f}")
print(f"European Call Price: ${european_price:.4f}")
print(f"\nAsian option is {((1 - asian_price/european_price)*100):.1f}% cheaper (due to averaging effect)")

## 4. Lookback Options

Lookback options depend on the maximum (or minimum) price achieved during the option's life.

In [ ]:
# Lookback option parameters
lookback_params = {
    'S0': 100.0,
    'K': 100.0,
    'r': 0.05,
    'delta': 0.02,
    'sigma': 0.25,
    'T': 1.0
}

# Generate 12-dimensional points for monthly monitoring
lookback_call_price = pricing.lookback_call(points=halton_12d, **lookback_params)
lookback_put_price = pricing.lookback_put(points=halton_12d, **lookback_params)

print(f"Lookback Call Price: ${lookback_call_price:.4f}")
print(f"Lookback Put Price: ${lookback_put_price:.4f}")
print(f"European Call Price: ${european_price:.4f}")
print(f"\nLookback call is {((lookback_call_price/european_price - 1)*100):.1f}% more expensive")

## 5. Comparison Summary

Let's create a summary table comparing all option types.

In [ ]:
summary = pd.DataFrame({
    'Option Type': ['European Call', 'Asian Call', 'Lookback Call', 'Spread (K=0)'],
    'Price ($)': [
        european_price,
        asian_price,
        lookback_call_price,
        margrabe_price
    ],
    'Relative to European': [
        '100%',
        f"{(asian_price/european_price*100):.1f}%",
        f"{(lookback_call_price/european_price*100):.1f}%",
        f"{(margrabe_price/european_price*100):.1f}%"
    ]
})

print("\nOption Pricing Summary:")
print(summary.to_string(index=False))